In [1]:
%pwd

'C:\\Users\\modong\\DCGAN'

In [2]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Reshape, Flatten, Dropout
from keras.layers.convolutional import UpSampling2D, Conv2D

from keras.optimizers import SGD, Adam
from keras.datasets import mnist
from keras.regularizers import l1_l2


C:\Users\modong\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# noise[[examples, 100]]から生成した画像をplot_dim(例えば4x4)で表示
def plot_generated(noise, generator_model, examples=16, plot_dim=(4,4), size=(7,7), epoch=None):
    # noiseからgeneratorで画像を生成
    generated_images = generator_model.predict(noise)

    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = generated_images[i, :]
        img = img.reshape((28, 56))
        plt.tight_layout()
        plt.imshow(img, cmap="gray")
        plt.axis("off")
    plt.subplots_adjust(wspace=0.2, hspace=0.2)
    plt.savefig(os.path.join("generated_figures", str(epoch) + ".png"))
    plt.close()

In [4]:
def plot_metrics(metrics, epoch=None):
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join("metrics", "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join("metrics", "g_loss" + str(epoch) + ".png"))
    plt.close()

In [5]:
def Generator():
    Gen = Sequential()
    Gen.add(Dense(input_dim=100, units=1024))
    Gen.add(BatchNormalization())
    Gen.add(Activation("relu"))
    Gen.add(Dense(units=128*7*7))
    Gen.add(BatchNormalization())
    Gen.add(Activation("relu"))
    Gen.add(Reshape((7,7,128), input_shape=(128*7*7,)))
    Gen.add(UpSampling2D((2,2)))
    Gen.add(Conv2D(64, 5, padding="same"))
    Gen.add(BatchNormalization())
    Gen.add(Activation("relu"))
    Gen.add(UpSampling2D((2,4)))
    Gen.add(Conv2D(1, 5, padding="same"))
    Gen.add(Activation("tanh"))
    generator_optimizer = SGD(lr=0.1, momentum=0.3, decay=1e-5)
    # Generatorは単独で訓練しないので下のcompileはいらない
    Gen.compile(loss="binary_crossentropy", optimizer=generator_optimizer)
    return Gen

In [6]:
def Discriminator():
    act = keras.layers.advanced_activations.LeakyReLU(alpha=0.2)
    Dis = Sequential()
    Dis.add(Conv2D(filters=64, kernel_size=(5, 5), strides=(2,2), padding="same", input_shape=(28,56,1)))
    Dis.add(act)
    Dis.add(Conv2D(filters=128, kernel_size=(5, 5), strides=(2,2), padding="same"))
    Dis.add(act)
    Dis.add(Flatten())
    Dis.add(Dense(units=256))
    Dis.add(act)
    Dis.add(Dropout(0.5))
    Dis.add(Dense(1))
    Dis.add(Activation("sigmoid"))
    discriminator_optimizer = Adam(lr=1e-5, beta_1=0.1)
    Dis.compile(loss="binary_crossentropy", optimizer=discriminator_optimizer)
    return Dis

In [7]:
def Generative_Adversarial_Network(generator_model, discriminator_model):
    GAN = Sequential()
    GAN.add(generator_model)
    discriminator_model.trainable=False
    GAN.add(discriminator_model)
    gan_optimizer = Adam(lr=1e-5, beta_1=0.1)
    GAN.compile(loss="binary_crossentropy", optimizer=gan_optimizer)
    return GAN

# ここからOpenCVによる画像読み込み。整形

In [8]:
import cv2
import numpy as np
X=[]
print

#for i in range(636):

for i in range(636):
    fn_img = "./output/"+str(i)+".bmp" 
    img = cv2.imread(fn_img,cv2.IMREAD_GRAYSCALE) 
    img=cv2.resize(img,(56,28))
    #print(img)

   
    X.append(img)

    #cv2.imshow("image", img)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    print(len(X))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
#list to float numpy
X= np.array(X, dtype=float)

In [10]:
# shuffle
import random

random.shuffle(X)

In [11]:
# separate
splitratio=0.8
num=int(len(X)*splitratio)

X_train=X[:num]
X_test=X[num:]
print(len(X_train))
print(len(X_test))

508
128


In [12]:

def  plot_metrics (metrics, epoch=None):
    plt.figure(figsize=(10,8))
    plt.plot(metrics["d"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join("metrics", "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join("metrics", "g_loss" + str(epoch) + ".png"))
    plt.close()

In [13]:
"""
(X_train, y_train), (X_test, y_test) = mnist.load_data()
"""


X_train.astype('float32')
X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255

print("X_train shape", X_train.shape)
print(X_train.shape[0], "train samples")
print(X_test.shape[0], "test samples")


X_train shape (508, 28, 56)
508 train samples
128 test samples


In [14]:
def main_train(z_input_size, generator_model, discriminator_model, gan_model, loss_dict, X_train, generated_figures=None, z_group=None, z_plot_freq=200, epoch=1000, plot_freq=25, batch=100):

    # tqdmでプログレスバー表示
    with tqdm(total=epoch) as pbar:
        for e in range(epoch):
            pbar.update(1)

            # 生成データをノイズから生成
            noise = np.random.uniform(0, 1, size=[batch, z_input_size])
            generated_images = generator_model.predict_on_batch(noise)

            # 訓練データをMNISTデータ群から抜粋
            rand_train_index = np.random.randint(0, X_train.shape[0], size=batch)
            image_batch = X_train[rand_train_index, :]

            # 訓練データと生成データを結合
            
            image_batch = np.expand_dims(image_batch, axis=3)
            #print(image_batch.shape)
            #print(generated_images.shape)
            X = np.vstack((image_batch, generated_images))
            # ラベル作成
            y = np.zeros(int(2*batch))
            y[batch:] = 1
            y = y.astype(int)

            # discriminatorの学習
            discriminator_model.trainable = True
            d_loss = discriminator_model.train_on_batch(x=X, y=y)
            discriminator_model.trainable = False

            # generatorの学習
            noise = np.random.uniform(0, 1, size=[batch, z_input_size])
            y = np.zeros(batch)
            y = y.astype(int)
            g_loss = gan_model.train_on_batch(x=noise, y=y)

            loss_dict["d"].append(d_loss)
            loss_dict["g"].append(g_loss)

            # グラフ描画
            if e%plot_freq == plot_freq-1:
                plot_metrics(loss_dict, int(e/plot_freq))
                generator_model.save('./model/gen_model_' +  str(int(e/z_plot_freq)) + '.h5')
                generator_model.save_weights('./model/gen_model_weights_' +  str(int(e/z_plot_freq)) + '.h5')
                gan_model.save('./model/gan_model_' + str(int(e/z_plot_freq)) + '.h5')
                gan_model.save_weights('./model/gan_model_weights_' +  str(int(e/z_plot_freq)) + '.h5')                

            # 訓練したgeneratorによる生成画像を可視化
            if e < epoch:
                if e%z_plot_freq == z_plot_freq-1:
                    plot_generated(z_group, generator_model=generator_model, epoch=int(e/z_plot_freq))
                    #generated_figures.append(fig)

In [15]:
# モデル生成
Gen = Generator()
Dis = Discriminator()
GAN = Generative_Adversarial_Network(Gen, Dis)
#GAN.summary()
#Gen.summary()
#Dis.summary()

# パラメータ設定
gan_losses = {"d":[], "g":[], "f":[]}
epoch = 100000
batch = 64
z_plot_freq = 500
plot_freq = 10000
z_input_vector = 100
n_train_samples = 60000
examples = 16


z_group_matrix = np.random.uniform(0,1,examples*z_input_vector)
z_group_matrix = z_group_matrix.reshape([16, z_input_vector])
print(z_group_matrix.shape)

generated_figures = []

a = Gen.predict_on_batch(z_group_matrix)
a.shape

(16, 100)


(16, 28, 56, 1)

In [ ]:

main_train (100, Gen, Dis, GAN, loss_dict=gan_losses, X_train=X_train, generated_figures=generated_figures, z_group=z_group_matrix, z_plot_freq=z_plot_freq, epoch=epoch, plot_freq=plot_freq, batch=batch)

 25%|█████████████████▊                                                     | 25045/100000 [1:18:12<8:52:47,  2.34it/s]

# 生成→OpenCVで保存部分